In [2]:

!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.4 MB/s eta 0:00:00a 0:00:01


In [3]:
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch -y
!pip install -U sentence-transformers

Retrieving notices: ...working... done
Channels:
 - pytorch
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package cuda-version-12.3-h55a0123_2 has constraint cudatoolkit 12.3|12.3.* conflicting with cudatoolkit-10.0.130-0

Could not solve for environment specs
The following packages are incompatible
├─ cuda-version 12.3**  is installable and it requires
│  └─ cudatoolkit 12.3|12.3.* , which can be installed;
└─ cudatoolkit 10.0**  is not installable because it conflicts with any installable versions previously reported.



In [4]:
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sentence_transformers import SentenceTransformer, util

# Load training data
with open("/kaggle/input/dataset-contractnli/train.json", "r") as f:
    train_data = json.load(f)

with open("/kaggle/input/dataset-contractnli/test.json", "r") as f:
    test_data = json.load(f)

with open("/kaggle/input/contractnli-dev/dev.json", "r") as f:
    val_data = json.load(f)

# Load BERT models
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
nli_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Load SentenceTransformer for semantic search
embedder = SentenceTransformer("all-MiniLM-L6-v2")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [6]:
def prepare_document_embeddings(documents):
    """Embed all sentences in each document for semantic search."""
    document_embeddings = []
    for doc in documents:
        sentences = doc.split('.')  # Adjust sentence splitting as needed
        embeddings = embedder.encode(sentences, convert_to_tensor=True,show_progress_bar=False)
        document_embeddings.append((sentences, embeddings))
    return document_embeddings

# Extract documents for embedding preparation
# train_data['documents'] = train_data['documents'][:5]
documents = [item['text'] for item in train_data['documents']]
document_embeddings = prepare_document_embeddings(documents)

documents = [item['text'] for item in val_data['documents']]
val_embeddings = prepare_document_embeddings(documents)

documents = [item['text'] for item in test_data['documents']]
test_embeddings = prepare_document_embeddings(documents)


In [7]:
label_dict = {'Entailment': 0, 'NotMentioned': 1, 'Contradiction': 2}

In [8]:
# Function to evaluate loss or accuracy
def evaluate(model, documents, labels, device,document_embeddings, mode="loss" ):
    total = 0
    correct = 0
    total_loss = 0
    model.eval()
    
    with torch.no_grad():
        cnt=0
        for idx, doc in enumerate(documents):
            doc_text = clean_text(doc['text'])
            annotations = doc['annotation_sets'][0]['annotations']
            document = document_embeddings[idx]  # Precomputed embeddings for the current document

            for hypothesis_name, annotation in annotations.items():
                choice = annotation['choice']
                hypothesis = clean_text(labels[hypothesis_name]['hypothesis'])
                label = label_dict[choice]
                
                # Retrieve top relevant sentence
                hypothesis_embedding = embedder.encode(hypothesis, convert_to_tensor=True, show_progress_bar=False).to(device)
                sentences, embeddings = document
                cos_scores = util.pytorch_cos_sim(hypothesis_embedding, embeddings.to(device))[0]
                top_result_idx = torch.argmax(cos_scores).item()
                top_sentence = sentences[top_result_idx]
                
                # Prepare input for BERT: "hypothesis <sep> result"
                input_text = f"{hypothesis} [SEP] {top_sentence}"
                inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
                inputs = {k: v.to(device) for k, v in inputs.items()}
                inputs['labels'] = torch.tensor([label], dtype=torch.long).to(device)

                
                # Forward pass
                outputs = model(**inputs)
                
                # print(input_text)
                # print(outputs )
                cnt+=1
                if mode == "loss":
                    loss = outputs.loss
                    total_loss += loss.item()
                elif mode == "accuracy":
                    predictions = torch.argmax(outputs.logits, dim=-1)
                    if predictions.item() == label:
                        correct += 1
                    total += 1

    if mode == "loss":
        return total_loss / cnt
    elif mode == "accuracy":
        return correct / total if total > 0 else 0

In [9]:
import torch
from transformers import AdamW
import re
from torch.nn.functional import softmax
from sentence_transformers import util

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Optimizer
optimizer = AdamW(nli_model.parameters(), lr=2e-5)

# Move models to device
nli_model = nli_model.to(device)
embedder = embedder.to(device)

# Prepare hypotheses dictionary
hypothesis_dict = {key: val['hypothesis'] for key, val in train_data['labels'].items()}

# Data and label dictionary
data = train_data
documents = data['documents']
val_documents = val_data['documents']
val_labels = val_data['labels']

test_documents = test_data['documents']
test_labels = test_data['labels']

labels = data['labels']

# Text cleaning function
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Training loop with validation loss
num_epochs = 2
top_k = 7
for epoch in range(num_epochs):
    nli_model.train()
    
    total_loss = 0
    cnt = 0
    
    for idx, doc in enumerate(documents):
        doc_text = clean_text(doc['text'])
        
        annotations = doc['annotation_sets'][0]['annotations']
        document = document_embeddings[idx]  # Precomputed embeddings for the current document

        for hypothesis_name, annotation in annotations.items():
            cnt += 1
            choice = annotation['choice']
            hypothesis = clean_text(labels[hypothesis_name]['hypothesis'])
            label = label_dict[choice]

            # Retrieve top 10 relevant sentences
            hypothesis_embedding = embedder.encode(hypothesis, convert_to_tensor=True, show_progress_bar=False).to(device)
            sentences, embeddings = document
            cos_scores = util.pytorch_cos_sim(hypothesis_embedding, embeddings.to(device))[0]
            top_k_results = torch.topk(cos_scores, k=top_k)
            top_sentences = [sentences[idx] for idx in top_k_results.indices]
            top_scores = top_k_results.values

            # Prepare inputs for BERT and compute weighted loss
            weighted_loss = 0
            weight_sum = torch.sum(top_scores).item()
            for i in range(top_k):
                input_text = f"{hypothesis} [SEP] {top_sentences[i]}"
                inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
                inputs['labels'] = torch.tensor([label], dtype=torch.long).to(device)
                
                # Move inputs to device
                inputs = {k: v.to(device) for k, v in inputs.items()}

                # Zero gradients
                optimizer.zero_grad()
                
                # Forward pass
                outputs = nli_model(**inputs)
                loss = outputs.loss
                
                # Weighted loss based on similarity score
                weighted_loss += (top_scores[i] / weight_sum) * loss

            total_loss += weighted_loss.item()
            
            # Backward pass and optimization step
            weighted_loss.backward()
            optimizer.step()

    print(f"Epoch {epoch + 1}, Training Loss: {total_loss / cnt}")

    # Validation Loss Calculation
    val_loss = evaluate(nli_model, val_documents, val_labels, device, val_embeddings, mode="loss")
    print(f"Epoch {epoch + 1}, Validation Loss: {val_loss}")

# Save fine-tuned model
nli_model.save_pretrained("fine_tuned_nli_model")
tokenizer.save_pretrained("fine_tuned_nli_model")




/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Training Loss: 0.7533996435647182
Epoch 1, Validation Loss: 0.7919737055768388
Epoch 2, Training Loss: 0.7293168594582806
Epoch 2, Validation Loss: 0.7771420771239923


('fine_tuned_nli_model/tokenizer_config.json',
 'fine_tuned_nli_model/special_tokens_map.json',
 'fine_tuned_nli_model/vocab.txt',
 'fine_tuned_nli_model/added_tokens.json')

In [10]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# Define evaluation function to compute metrics
def evaluate_metrics(nli_model, documents, labels, device, embeddings, top_k=7):
    nli_model.eval()
    all_true_labels = []
    all_pred_labels = []
    
    with torch.no_grad():
        for idx, doc in enumerate(documents):
            document = embeddings[idx]  # Precomputed embeddings for the current document
            annotations = doc['annotation_sets'][0]['annotations']
            
            for hypothesis_name, annotation in annotations.items():
                choice = annotation['choice']
                true_label = label_dict[choice]
                hypothesis = clean_text(labels[hypothesis_name]['hypothesis'])
                
                # Retrieve top-k relevant sentences
                hypothesis_embedding = embedder.encode(hypothesis, convert_to_tensor=True, show_progress_bar=False).to(device)
                sentences, sentence_embeddings = document
                cos_scores = util.pytorch_cos_sim(hypothesis_embedding, sentence_embeddings.to(device))[0]
                top_k_results = torch.topk(cos_scores, k=top_k)
                top_sentences = [sentences[idx] for idx in top_k_results.indices]
                top_scores = top_k_results.values

                # Weighted average for label prediction
                weighted_logits_sum = torch.zeros((1, nli_model.num_labels)).to(device)
                for i in range(top_k):
                    input_text = f"{hypothesis} [SEP] {top_sentences[i]}"
                    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)
                    
                    # Forward pass
                    outputs = nli_model(**inputs)
                    logits = outputs.logits
                    weighted_logits_sum += top_scores[i] * logits
                
                # Calculate prediction by taking the label with max probability
                avg_logits = weighted_logits_sum / torch.sum(top_scores)
                predicted_label = torch.argmax(softmax(avg_logits, dim=-1), dim=-1).item()
                
                # Append to true and predicted labels
                all_true_labels.append(true_label)
                all_pred_labels.append(predicted_label)

    # Calculate metrics
    accuracy = (torch.tensor(all_pred_labels) == torch.tensor(all_true_labels)).float().mean().item()
    precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_pred_labels, average='weighted')
    conf_matrix = confusion_matrix(all_true_labels, all_pred_labels)

    # Print results
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    return accuracy, precision, recall, f1, conf_matrix

# Evaluate metrics
top_k = 7
accuracy, precision, recall, f1, conf_matrix = evaluate_metrics(
    nli_model, test_documents, test_labels, device, test_embeddings, top_k=top_k
)


Accuracy: 61.41%
Precision: 0.6377
Recall: 0.6141
F1 Score: 0.5869
Confusion Matrix:
[[864  98   6]
 [542 332  29]
 [ 70  62  88]]


In [11]:
import torch
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util



# Evidence identification function
def identify_evidence(document, hypothesis, nli_model, tokenizer, embedder, device):
    # Step 1: Split document into sentences
    sentences = document.split('. ')  # Split sentences based on period followed by a space
    
    # Step 2: Perform NLI classification for the hypothesis and document
    input_text = f"{hypothesis} [SEP] {document}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = nli_model(**inputs)
    logits = outputs.logits
    probs = softmax(logits, dim=-1)
    nli_class = torch.argmax(probs).item()
    
    class_names = ["entailment", "neutral", "contradiction"]
    predicted_class = class_names[nli_class]

    # Step 3: Find the sentence that supports the predicted class with the highest probability
    hypothesis_embedding = embedder.encode(hypothesis, convert_to_tensor=True).to(device)
    sentence_embeddings = embedder.encode(sentences, convert_to_tensor=True).to(device)
    cos_scores = util.pytorch_cos_sim(hypothesis_embedding, sentence_embeddings).squeeze(0)
    
    max_score_idx = torch.argmax(cos_scores).item()
    supporting_sentence = sentences[max_score_idx]

    # Step 4: Return results
    return {
        "predicted_class": predicted_class,
        "supporting_sentence": supporting_sentence,
        "supporting_score": cos_scores[max_score_idx].item()
    }

# Example usage
document = """
The contract states that the payment must be made within 30 days of delivery.
If the payment is delayed, a penalty of 2% per month is applicable.
The client also agrees to cover all shipping costs.
"""
hypothesis = "The client is responsible for shipping costs."

result = identify_evidence(document, hypothesis, nli_model, tokenizer, embedder, device)

# Output the results
print("Predicted Class:", result['predicted_class'])
print("Supporting Sentence:", result['supporting_sentence'])
print("Supporting Score:", result['supporting_score'])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Predicted Class: entailment
Supporting Sentence: 
The contract states that the payment must be made within 30 days of delivery.
If the payment is delayed, a penalty of 2% per month is applicable.
The client also agrees to cover all shipping costs.

Supporting Score: 0.48570507764816284
